# Risk Sensative Control

## Introdunction
Exisiting autonoumous driving controls assume that the users of the autonoumous driving system prefer the similar pattern, in practical scenarios, this is rarely the case. This paper introduces Risk Sensative Control (RSC), an inverse optimal control algorithm that is able to estimate the risk sensative driving features, and incorporate those features online into a receding-horizon cotroller. RSC solves the inverse optimal control problem by reducing subjective risk perseption from users feedback. RSC uses an Meta-learning algorithm to update the parameters of the cost function parameters, and  improve the controller online as more data is gathered from the users subjective risk feedback. The estimater take acounts of the uncertainty in the subjective risk analysis in driving features and surroudning vehicle locations, by uncertainty ellipse constraints. This approach is verified in realtime in five lane change both safe and risky scenarios in CARLA simulation environments.


## Related Work

### Subjective Risk Estimation

### Objective Function Estimation
Estimating objective functions from historical data is a well investigated problem known as Inverse Optimal Control
(IOC) or Inverse Reinforcement Learning (IRL). Typically, with the IRL approach, the objective function is linear in
terms of a given set of state features. The goal is to identify a parameter vector that weights these features so that
the behavior resulting from this estimated objective matches the observed behavior. 
Optimal control methods are optimizers, but in reality, we don’t really know what we should be optimizing. The question of how we can, instead, leverage our strong intuition of how we’d like the robot to behave to learn a cost function that embodies demonstrations of that behavior. So defines Inverse Optimal Control (IOC). Rather than computing an optimal policy given a cost function, IOC uncovers the cost function that best explains demonstrated optimal behavior. 

### Online Parameter Estimation
We review approaches proposed for online parameter estimation in this context. The Intelligent Driver Model (IDM), is a rule-based lane-following model balancing two objectives: reaching a desired speed and avoiding collision with
the preceding car. This model has few tunable parameters and returns the scalar acceleration of the vehicle along a predefined
path. Several works used online filtering techniques to estimate these IDM parameters.


### Data-Driven driving Control
There is a rich literature on applying data-driven approaches
to pedestrian or vehicle trajectory predictions. Such approaches usually require a large corpus of data and are trained
offline as a general model to suit multiple agents. On the other hand, we require a small amount of data and find parameters
online for a specific agent. Data-driven methods can predict a distribution over future trajectories e.g., offline inverse optimal
control with online goal inference; Conditional Variational Autoencoders (CVAE) or Generative Adversarial Networks (GAN). 

## Problem formulation

- Ego vehicle state: $$z=\begin{bmatrix}x\\ y\\ \phi\\ v\\ \omega\\  l \\ \dot{a} \\ \ddot{a}\end{bmatrix}$$

which represents $x, y$ position of trajectory, yaw angle, velocity, lateral acceleration, longitudinal acceleration, derivative of accel force, second order derivative of accel force, derivative of brake force, second order derivative of brake force.

- Ego vehicle control $$u = \begin{bmatrix}\delta\\a\end{bmatrix}$$

which represents steering angle, accel force, or brake force.
 

In [1]:
import math
import numpy as np
import scipy.interpolate

In [ ]:
# PARAMETER
L = 2.5  # wheel base
ds = 0.1  # course distanse
TARGET_SPEED = 30.0 / 3.6  # velocity [m/s]


In [3]:
class State:
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0, omega=0.0, la=0.0, da=0.0, dda=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.omega = omega
        self.la = la
        self.da = da
        self.dda = dda
        

In [4]:
class Input:
    def __init__(self, delta=0, a=0):
        self.delta = 0
        self.a = 0
        

## Problem Formulation

## Features and Objectives

Since we use CARLA simulation environment to demenstrate our experiment, we list up the features information as following.



Here is the rotation definition in CARLA.

<img src=./fig/carla_rotation.jpg width="500px" >

In [1]:
import os
import glob
import sys
from carla import ColorConverter as cc

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

In [2]:
class ModelPredictiveControl(object):
    def __init__(self, world, target_traj, start_in_autopilot):
        self._autopilot_enabled = start_in_autopilot # Keep autopilot
        self.target_traj = target_traj
        
        self._current_x          = 0
        self._current_y          = 0
        self._current_yaw        = 0
        self._current_speed      = 0
        self._desired_speed      = 0
        self._current_frame      = 0
        self._current_timestamp  = 0
        self._start_control_loop = False
        self._set_throttle       = 0
        self._set_brake          = 0
        self._set_steer          = 0
        self._waypoints          = waypoints
        self._conv_rad_to_steer  = 180.0 / 70.0 / np.pi
        self._pi                 = np.pi
        self._2pi                = 2.0 * np.pi
        
    def update_values(self, x, y, yaw, speed, timestamp, frame):
        self._current_x         = x
        self._current_y         = y
        self._current_yaw       = yaw
        self._current_speed     = speed
        self._current_timestamp = timestamp
        self._current_frame     = frame
        if self._current_frame:
            self._start_control_loop = True
    
    def parse_events(self, world, clock):
        if not self._autopilot_enabled:
            if isinstance(self._control, carla.VehicleControl):carla.Location(carla.Vector3D), 
                self._parse_vehicle_keys(pygame.key.get_pressed(), clock.get_time())
                self._parse_vehicle_wheel()
                self._control.reverse = self._control.gear < 0
            elif isinstance(self._control, carla.WalkerControl):
                self._parse_walker_keys(pygame.key.get_pressed(), clock.get_time())
            
            self._parse_naren_control(world)
            world.player.apply_control(self._control)
        
        
    def _parse_naren_control(self, world):
        [lc_x, lc_y, lc_yaw] = self.target_traj
        print('Here we can get mpc control in each time step\n')
        
        t = world.player.get_transform() # Return carla.Transform(carla.Vector3D), carla.Rotation(pitch(Y),yaw(Z), roll(X))[degree] 
        location = world.player.get_location #Return: carla.Location – meters
        v = world.player.get_velocity()  # Return: carla.Vector3D – m/s,v.x,v.y.v.z
        c = world.player.get_control()   # Return: carla.VehicleControl:throttle:[0,1], steer:[-1,1], brake:[0,1]
        a = world.player.get_acceleration() #Return: carla.Vector3D – m/s2
        w = world.player.get_angular_velocity() #Return: carla.Vector3D – deg/s
        
        
        

IndentationError: unexpected indent (<ipython-input-2-481843a2f4c8>, line 11)


### Risk Sensative Motion Model

We focus on the discretized dynamic state-space form setting within length $N$ time horizon. We denote $z_t \in R^n$ the state and $u_t \in R^m$ the control input of the ego vehicle in $t \in \{1,...,N\}$ time steps. Ego vehicle's strategy is a control input sequence $U = [u_{t+1}, u_{t+2}, ...,u_{t+N}]$ and $u_{t+1}$ will be applied in each loop. 

The vehicle dynamic is shown in this figure.
<img src=motion_model.png width="200px" >

We define our motion model as follows:
$$
\begin{eqnarray}
\dot{x} &=& v cos(\phi) \\
\dot{y} &=& v sin(\phi)\\
\dot{v} &=& a\\
\dot{\phi} &=& \frac{v}{L} tan(\delta)\\
\dot{\omega} &=&  a sin(\phi-\delta)\\
\dot{l}&=&  a cos(\phi-\delta)\\
\dot{a} &=& a dt\\
\ddot{a} &=& \dot{a} dt\\
\end{eqnarray}
$$
where $L$ is the length between the front and reer wheel.

It stems from executing the control strategies of all the surrouding vehicles on a joint dyanamical systems.
$$ z_{t+1} = A z_{t} + B u_t +C \\
\dot{z} = f(z,u) = A'z + B'u  
$$

ODE can be witten as
$$
\frac{\partial f}{\partial z} =
A'= 
\begin{bmatrix}
0&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&0& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&0&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&0&0&0&1&0
\end{bmatrix}
$$

$$
\frac{\partial f}{\partial u} =
B' = 
\begin{bmatrix}
0&0\\
0&0\\
0&1\\
\frac{v}{L cos^2(\delta)}&0\\
-a cos(\phi-\delta)&sin(\phi-\delta)\\
-a sin(\phi-\delta)&cos(\phi-\delta)\\
0&dt\\
0&0\\
\end{bmatrix}
$$

We can get a discrete-time mode with Forward Euler Discretization with sampling time $dt$.

$$z_{t+1} = z_k + f(z_t, u_t)dt$$

Using first degree Tayler expantion around $\bar{z}$ and $\bar{u}$
$$z_{t+1} = z_t + (f({\bar{z}, \bar{u}}) + A'z_t + B'u_t-A'\bar{z}-B'\bar{u})dt$$
$$z_{t+1} = (I+dtA')z_t + (dtB')u_t + (f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u}))dt$$

Therefore, we get
$$
A= 
\begin{bmatrix}
1&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&1& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 1&0&0&0&0&0\\
0&0& 0&1+tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&1&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&1&0&0\\
0&0& 0&0&0&&1&0\\
0&0& 0&0&0&0&1&1
\end{bmatrix}
$$

$$
B = 
\begin{bmatrix}
0&0\\
0&0\\
0&dt\\
dt\frac{v}{L*cos^2(\delta)}&0\\
-a*cos(\phi-\delta)dt&sin(\phi-\delta)dt\\
-a*sin(\phi-\delta)dt&cos(\phi-\delta)dt\\
0&dt\\
0&0\\
\end{bmatrix}
$$

$$
f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u}) = \\
\begin{bmatrix}
\bar{v} cos(\bar{\phi}) \\
\bar{v} sin(\bar{\phi})\\
\bar{a}\\
\frac{\bar{v}}{L} tan(\bar{\delta})\\
\bar{a}sin(\bar{\phi}-\bar{\delta})\\
\bar{a}cos(\bar{\phi}-\bar{\delta})\\
\bar{a}dt\\
\bar{\dot{a}}dt\\
\end{bmatrix}
-
\begin{bmatrix}
0&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&0& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&0&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&0&0&0&1&0
\end{bmatrix}
\begin{bmatrix}\bar{x}\\ \bar{y}\\ \bar{\phi}\\ \bar{v}\\ \bar{\omega}\\  \bar{l} \\ \bar{\dot{a}} \\ \bar{\ddot{a}}\end{bmatrix}-
\begin{bmatrix}
0&0\\
0&0\\
0&1\\
\frac{v}{L*cos^2(\delta)}&0\\
-a*cos(\phi-\delta)&sin(\phi-\delta)\\
-a*sin(\phi-\delta)&cos(\phi-\delta)\\
0&dt\\
0&0\\
\end{bmatrix}
\begin{bmatrix}\bar{\delta}\\\bar{a}\end{bmatrix}$$


$$=
\begin{bmatrix}
\bar{v} cos(\bar{\phi}) \\
\bar{v} sin(\bar{\phi})\\
\bar{a}\\
\frac{\bar{v}}{L} tan(\bar{\delta})\\
\bar{a}sin(\bar{\phi}-\bar{\delta})\\
\bar{a}cos(\bar{\phi}-\bar{\delta})\\
\bar{a}dt\\
\bar{\dot{a}}dt\\
\end{bmatrix} -
\begin{bmatrix}
-\bar{v}*sin(\bar{\phi})\bar{\phi} +\bar{v}cos(\bar{\phi})\\
 \bar{v}*cos(\bar{\phi})\bar{\phi} +\bar{v}sin(\bar{\phi})\\
0\\
tan({\bar\delta})\bar{v}/L\\
\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}\\
-\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi} \\
0\\
0
\end{bmatrix}
-
\begin{bmatrix}
0\\
0\\
\bar{a}\\
\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta} + sin(\bar{\phi}-\bar{\delta})\bar{a} \\
-\bar{v}\bar{\delta}sin(\bar{\phi}-\bar{\delta})\bar{\delta}+ cos(\bar{\phi}-\bar{\delta})\bar{a}\\
\bar{a}dt\\
0
\end{bmatrix}
=
\begin{bmatrix}
\bar{v}*sin(\bar{\phi})\bar{\phi} \\
-\bar{v}*cos(\bar{\phi})\bar{\phi}\\
0\\
-\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
\bar{\delta})-\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}-\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta}\\
-\bar{\delta})+\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi}+\bar{v}\bar{\delta}sin(\bar{\phi}+ \bar{\delta})\bar{\delta} \\
0\\
\bar{\dot{a}}dt\\
\end{bmatrix}
$$

$$
C = \begin{bmatrix}
\bar{v}*sin(\bar{\phi})\bar{\phi} \\
-\bar{v}*cos(\bar{\phi})\bar{\phi}\\
0\\
-\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
-\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}-\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta}\\
\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi}+\bar{v}\bar{\delta}sin(\bar{\phi}+ \bar{\delta})\bar{\delta} \\
0\\
\bar{\dot{a}}dt\\
\end{bmatrix}dt
$$


In a mylti-player driving situation, we assume the other agents are 'ideal' players. We further assume that ego vehicle and surrounding ego vehicles in a receding horizon loop.


### Objective Function Parameterization

$$
$$

$$ Contranits:
- Distance to surrounding vehicle


$$

In [ ]:
def main():
    argparser = argparse.ArgumentParser(
        description='CARLA Personalized Model Predictive Control Client')
    argparser.add_argument(
        '-v', '--verbose',
        action='store_true',
        dest='debug',
        help='print debug information')
    argparser.add_argument(
        '--host',
        metavar='H',
        default='127.0.0.1',
        help='IP of the host server (default: 127.0.0.1)')
    argparser.add_argument(
        '-p', '--port',
        metavar='P',
        default=2000,
        type=int,
        help='TCP port to listen to (default: 2000)')
    argparser.add_argument(
        '-a', '--autopilot',
        action='store_true',
        help='enable autopilot')
    argparser.add_argument(
        '--res',
        metavar='WIDTHxHEIGHT',
        default='1280x720',
        help='window resolution (default: 1280x720)')
    argparser.add_argument(
        '--filter',
        metavar='PATTERN',
        default='vehicle.*',
        help='actor filter (default: "vehicle.*")')
    args = argparser.parse_args()

    args.width, args.height = [int(x) for x in args.res.split('x')]

    log_level = logging.DEBUG if args.debug else logging.INFO
    logging.basicConfig(format='%(levelname)s: %(message)s', level=log_level)

    logging.info('listening to server %s:%s', args.host, args.port)

    print(__doc__)

    try:
        game_loop(args)

    except KeyboardInterrupt:
        print('\nCancelled by user. Bye!')

In [ ]:
if __name__ == '__main__':
    main()    

In [8]:
def update(prev_state, delta, a, b, dt, L):

    x_t = prev_state.x
    y_t = prev_state.y
    yaw_t = prev_state.yaw
    v_t = prev_state.v
    omega_t = prev_state.omega
    la_t = prev_state.la
    da_t = prev_state.da
    dda_t = prev_state.dda
    db_t = prev_state.db
    ddb_t = prev_state.ddb
    
    delta_t = delta
    if a>0 and b==0:
        a_t = a
        da_t_dot = a_t*dt
        dda_t_dot = da_t *dt
        db_t_dot = 0
        ddb_t_dot = 0
    elif a==0 and b>0:
        a_t = b
        da_t_dot = 0
        dda_t_dot = 0
        db_t_dot = a_t*dt
        ddb_t_dot = db_t*dt
    elif a==0 and b==0:
        a_t = 0
        da_t_dot = 0
        dda_t_dot = 0
        db_t_dot = 0
        ddb_t_dot = 0
    else:
        print('There is something wrong with the pedal force')
        
    x_dot = v_t *np.cos(yaw_t)
    y_dot = v_t*np.sin(yaw_t)
    v_t_dot = a_t
    yaw_t_dot = v_t*np.tan(delta_t)/L
    
    omega_t_dot = a_t*np.sin(yaw_t-delta_t)
    la_t_dot = a_t*np.cos(yaw_t-delta_t)
    
    
    state.x = x_t + x_dot*dt
    state.y = y_t + y_dot*dt
    state.yaw = pi_2_pi((yaw_t + yaw_t_dot)*dt)
    state.v = v_t + v_t_dot*dt
    state.omega = omega_t + omega_t_dot*dt
    state.la = la_t + la_t_dot*dt
    state.da = da_t + da_t_dot*dt
    state.db = db_t + db_t_dot*dt
    state.dda = dda_t + dda_t_dot*dt
    state.ddb = ddb_t + ddb_t_dot*dt

    return state

In [3]:
def pi_2_pi(angle):
    return (angle + math.pi) % (2 * math.pi) - math.pi

In [9]:
def get_lane_change_course_switch(dl):
    path_len = 120
    reso = 0.1
    cx = np.arange(0, path_len, reso)
    cy = np.array([2] * int(path_len / reso))
    cyaw = np.array([0.0] * int(path_len / reso))

    return cx, cy, cyaw, []

In [19]:
def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)

    return A, B, C


def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", truckcolor="-k"):  # pragma: no cover

    outline = np.array([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                        [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.array([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                         [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.array([[math.cos(yaw), math.sin(yaw)],
                     [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.array([[math.cos(steer), math.sin(steer)],
                     [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T.dot(Rot2)).T
    fl_wheel = (fl_wheel.T.dot(Rot2)).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T.dot(Rot1)).T
    fl_wheel = (fl_wheel.T.dot(Rot1)).T

    outline = (outline.T.dot(Rot1)).T
    rr_wheel = (rr_wheel.T.dot(Rot1)).T
    rl_wheel = (rl_wheel.T.dot(Rot1)).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    plt.plot(np.array(outline[0, :]).flatten(),
             np.array(outline[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fr_wheel[0, :]).flatten(),
             np.array(fr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rr_wheel[0, :]).flatten(),
             np.array(rr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fl_wheel[0, :]).flatten(),
             np.array(fl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rl_wheel[0, :]).flatten(),
             np.array(rl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(x, y, "*")




def get_nparray_from_matrix(x):
    return np.array(x).flatten()


def calc_nearest_index(state, cx, cy, cyaw, pind):
    """

    :param state:
    :param cx:
    :param cy:
    :param cyaw:
    :param pind: previous index
    :return: the index of the closest ref point and the distance to it
    """

    # obtain the vector from current state to next 10 ref points
    dx = [state.x - icx for icx in cx[pind:(pind + N_IND_SEARCH)]]
    dy = [state.y - icy for icy in cy[pind:(pind + N_IND_SEARCH)]]

    # calculate square distance
    d = [idx ** 2 + idy ** 2 for (idx, idy) in zip(dx, dy)]
    # obtain min square distance
    mind = min(d)
    # update current index
    ind = d.index(mind) + pind
    # obtain
    mind = math.sqrt(mind)

    dxl = cx[ind] - state.x
    dyl = cy[ind] - state.y

    angle = pi_2_pi(cyaw[ind] - math.atan2(dyl, dxl))
    if angle < 0:
        mind *= -1

    return ind, mind


def predict_motion(x0, oa, od, xref):
    xbar = xref * 0.0
    for i, _ in enumerate(x0):
        xbar[i, 0] = x0[i]

    state = State(x=x0[0], y=x0[1], yaw=x0[3], v=x0[2])
    for (ai, di, i) in zip(oa, od, range(1, T + 1)): # apply control from 1 to T
        state = update_state(state, ai, di)
        xbar[0, i] = state.x
        xbar[1, i] = state.y
        xbar[2, i] = state.v
        xbar[3, i] = state.yaw

    return xbar


def iterative_linear_mpc_control(xref, x0, dref, oa, od):
    """
    MPC contorl with updating operational point iteraitvely
    """

    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T

    # oa = [0.0] * T
    # od = [0.0] * T

    for i in range(MAX_ITER):
        print('iteration', i)
        xbar = predict_motion(x0, oa, od, xref)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, x0, dref)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")

    return oa, od, ox, oy, oyaw, ov, xbar


def linear_mpc_control(xref, xbar, x0, dref):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], dref[0, t])
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t] + C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <= MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == x0]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = get_nparray_from_matrix(x.value[0, :])
        oy = get_nparray_from_matrix(x.value[1, :])
        ov = get_nparray_from_matrix(x.value[2, :])
        oyaw = get_nparray_from_matrix(x.value[3, :])
        oa = get_nparray_from_matrix(u.value[0, :])
        odelta = get_nparray_from_matrix(u.value[1, :])

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov


def calc_ref_trajectory(state, cx, cy, cyaw, ck, sp, dl, pind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    ind, _ = calc_nearest_index(state, cx, cy, cyaw, pind)

    if pind >= ind:
        ind = pind

    xref[0, 0] = cx[ind]
    xref[1, 0] = cy[ind]
    xref[2, 0] = sp[ind]
    xref[3, 0] = cyaw[ind]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        # travel += abs(state.v) * DT
        travel += TARGET_SPEED * DT
        dind = int(round(travel / dl))

        if (ind + dind) < ncourse:
            xref[0, i] = cx[ind + dind]
            xref[1, i] = cy[ind + dind]
            xref[2, i] = sp[ind + dind]
            xref[3, i] = cyaw[ind + dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = cx[ncourse - 1]
            xref[1, i] = cy[ncourse - 1]
            xref[2, i] = sp[ncourse - 1]
            xref[3, i] = cyaw[ncourse - 1]
            dref[0, i] = 0.0

    return xref, ind, dref


def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False


def IDM_model(x_s_all, v_s_all, ego_veh_state):
    """
    :param x_s_all: positions and velocities of all surrounding vehicles, for example, when there are 3 vehicles
      [[x1, v1], [x2, v2], [x3, v3]]
    :return:
    """
    x_s_all_next = []
    v_s_all_next = []

    T = 1.5 # Safe time headway
    a = 0.73 # Maximum acceleration m/s^2
    b = 1.67 # Comfortable deceleration m/s^2
    delta = 4 # Acceleration exponent
    s0 = 2 # Minimum distance m

    # loop each vehicle
    for veh in range(len(x_s_all)):
        # calculate distance (TTC) to other vehicles, including ego vehicle
        # apply IDM model
        x = x_s_all[veh]
        v = v_s_all[veh]
        # print(ego_veh_state.x)
        # TODO: modify here: s is the longitudinal net distance to the vehicle in front and in the same lane
        s = math.sqrt((x-ego_veh_state.x)**2 + ego_veh_state.y**2)
        # TODO: you also need to consider

        s_star = s0 + v*T + (v*v*DT)/(2*math.sqrt(a*b))
        dx = DT * v
        dv = DT * a*(1-(v/TARGET_SPEED)**delta-(s_star/s)**2)

        x_s_all_next.append(x+dx)
        v_s_all_next.append(v+dv)

    return x_s_all_next, v_s_all_next

In [12]:
def calc_speed_profile(cx, cy, cyaw, target_speed):

    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile

In [20]:
def main():
    print(__file__ + " start!!")

    # dl = 1.0  # course tick
    # dl = TARGET_SPEED * DT
    dl = 0.1
    print(dl)
    print('traget speed:', TARGET_SPEED)
    # cx, cy, cyaw, ck = get_straight_course(dl)
    # cx, cy, cyaw, ck = get_straight_course2(dl)
    # cx, cy, cyaw, ck = get_straight_course3(dl)
    # cx, cy, cyaw, ck = get_forward_course(dl)
    # cx, cy, cyaw, ck = get_switch_back_course(dl)
    cx, cy, cyaw, ck = get_lane_change_course_switch(dl)

    sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)

    initial_state = State(x=cx[0], y=cy[0]-4, yaw=cyaw[0], v=TARGET_SPEED, omega=0.0, la=0.0, da=0.0, dda=0.0, db=0.0, ddb=0.0)

    # TODO 1: from previous step
    # Lane change number #1
    initial_state_surrounding = []

    # initial condition of surrounding vehicles:
    # 1st component: desired speed difference from 22.22 (80 km/h),
    # 2nd component: relative longitudinal distance (x) from ego vehicle (ego vehicle starts at x = 0)
    # 3rd component: not important
    # 4th component: lateral distance, which represents left or right lane (left lane: 2 right lane: -2)
    surrounding_veh_state_1 = [-0.8, 42.4, 0, 2] # left lane
    surrounding_veh_state_2 = [-0.8, 17, 0, -2] # right lane
    surrounding_veh_state_3 = [-0.8, -10, 0, -2]

    initial_state_surrounding.append(surrounding_veh_state_1)
    initial_state_surrounding.append(surrounding_veh_state_2)
    initial_state_surrounding.append(surrounding_veh_state_3)

    # t, x, y, yaw, v, d, a = do_simulation(
    #     cx, cy, cyaw, ck, sp, dl, initial_state)

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state, initial_state_surrounding)

    if show_animation:  # pragma: no cover
        plt.close("all")
        plt.subplots()
        plt.plot(cx, cy, "-r", label="spline")
        plt.plot(x, y, "-g", label="tracking")
        plt.grid(True)
        plt.axis("equal")
        plt.xlabel("x[m]")
        plt.ylabel("y[m]")
        plt.legend()

        plt.subplots()
        plt.plot(t, v, "-r", label="speed")
        plt.grid(True)
        plt.xlabel("Time [s]")
        plt.ylabel("Speed [kmh]")

        v_kmh = list(np.array(v) * 3.6)


        con_mpc = {'x': x, 'y': y, 'v': v_kmh, 'acc': a, 'steer': d, 'yaw': yaw}
        with open(pickle_file, mode='wb') as fo:
            pickle.dump(con_mpc, fo)


        plt.show()

In [18]:
def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)

    return A, B, C


def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", truckcolor="-k"):  # pragma: no cover

    outline = np.array([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                        [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.array([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                         [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.array([[math.cos(yaw), math.sin(yaw)],
                     [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.array([[math.cos(steer), math.sin(steer)],
                     [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T.dot(Rot2)).T
    fl_wheel = (fl_wheel.T.dot(Rot2)).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T.dot(Rot1)).T
    fl_wheel = (fl_wheel.T.dot(Rot1)).T

    outline = (outline.T.dot(Rot1)).T
    rr_wheel = (rr_wheel.T.dot(Rot1)).T
    rl_wheel = (rl_wheel.T.dot(Rot1)).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    plt.plot(np.array(outline[0, :]).flatten(),
             np.array(outline[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fr_wheel[0, :]).flatten(),
             np.array(fr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rr_wheel[0, :]).flatten(),
             np.array(rr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fl_wheel[0, :]).flatten(),
             np.array(fl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rl_wheel[0, :]).flatten(),
             np.array(rl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(x, y, "*")




def get_nparray_from_matrix(x):
    return np.array(x).flatten()


def calc_nearest_index(state, cx, cy, cyaw, pind):
    """

    :param state:
    :param cx:
    :param cy:
    :param cyaw:
    :param pind: previous index
    :return: the index of the closest ref point and the distance to it
    """

    # obtain the vector from current state to next 10 ref points
    dx = [state.x - icx for icx in cx[pind:(pind + N_IND_SEARCH)]]
    dy = [state.y - icy for icy in cy[pind:(pind + N_IND_SEARCH)]]

    # calculate square distance
    d = [idx ** 2 + idy ** 2 for (idx, idy) in zip(dx, dy)]
    # obtain min square distance
    mind = min(d)
    # update current index
    ind = d.index(mind) + pind
    # obtain
    mind = math.sqrt(mind)

    dxl = cx[ind] - state.x
    dyl = cy[ind] - state.y

    angle = pi_2_pi(cyaw[ind] - math.atan2(dyl, dxl))
    if angle < 0:
        mind *= -1

    return ind, mind


def predict_motion(x0, oa, od, xref):
    xbar = xref * 0.0
    for i, _ in enumerate(x0):
        xbar[i, 0] = x0[i]

    state = State(x=x0[0], y=x0[1], yaw=x0[3], v=x0[2])
    for (ai, di, i) in zip(oa, od, range(1, T + 1)): # apply control from 1 to T
        state = update_state(state, ai, di)
        xbar[0, i] = state.x
        xbar[1, i] = state.y
        xbar[2, i] = state.v
        xbar[3, i] = state.yaw

    return xbar


def iterative_linear_mpc_control(xref, x0, dref, oa, od):
    """
    MPC contorl with updating operational point iteraitvely
    """

    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T

    # oa = [0.0] * T
    # od = [0.0] * T

    for i in range(MAX_ITER):
        print('iteration', i)
        xbar = predict_motion(x0, oa, od, xref)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, x0, dref)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")

    return oa, od, ox, oy, oyaw, ov, xbar


def linear_mpc_control(xref, xbar, x0, dref):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], dref[0, t])
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t] + C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <= MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == x0]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = get_nparray_from_matrix(x.value[0, :])
        oy = get_nparray_from_matrix(x.value[1, :])
        ov = get_nparray_from_matrix(x.value[2, :])
        oyaw = get_nparray_from_matrix(x.value[3, :])
        oa = get_nparray_from_matrix(u.value[0, :])
        odelta = get_nparray_from_matrix(u.value[1, :])

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov


def calc_ref_trajectory(state, cx, cy, cyaw, ck, sp, dl, pind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    ind, _ = calc_nearest_index(state, cx, cy, cyaw, pind)

    if pind >= ind:
        ind = pind

    xref[0, 0] = cx[ind]
    xref[1, 0] = cy[ind]
    xref[2, 0] = sp[ind]
    xref[3, 0] = cyaw[ind]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        # travel += abs(state.v) * DT
        travel += TARGET_SPEED * DT
        dind = int(round(travel / dl))

        if (ind + dind) < ncourse:
            xref[0, i] = cx[ind + dind]
            xref[1, i] = cy[ind + dind]
            xref[2, i] = sp[ind + dind]
            xref[3, i] = cyaw[ind + dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = cx[ncourse - 1]
            xref[1, i] = cy[ncourse - 1]
            xref[2, i] = sp[ncourse - 1]
            xref[3, i] = cyaw[ncourse - 1]
            dref[0, i] = 0.0

    return xref, ind, dref


def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False


def IDM_model(x_s_all, v_s_all, ego_veh_state):
    """
    :param x_s_all: positions and velocities of all surrounding vehicles, for example, when there are 3 vehicles
      [[x1, v1], [x2, v2], [x3, v3]]
    :return:
    """
    x_s_all_next = []
    v_s_all_next = []

    T = 1.5 # Safe time headway
    a = 0.73 # Maximum acceleration m/s^2
    b = 1.67 # Comfortable deceleration m/s^2
    delta = 4 # Acceleration exponent
    s0 = 2 # Minimum distance m

    # loop each vehicle
    for veh in range(len(x_s_all)):
        # calculate distance (TTC) to other vehicles, including ego vehicle
        # apply IDM model
        x = x_s_all[veh]
        v = v_s_all[veh]
        # print(ego_veh_state.x)
        # TODO: modify here: s is the longitudinal net distance to the vehicle in front and in the same lane
        s = math.sqrt((x-ego_veh_state.x)**2 + ego_veh_state.y**2)
        # TODO: you also need to consider

        s_star = s0 + v*T + (v*v*DT)/(2*math.sqrt(a*b))
        dx = DT * v
        dv = DT * a*(1-(v/TARGET_SPEED)**delta-(s_star/s)**2)

        x_s_all_next.append(x+dx)
        v_s_all_next.append(v+dv)

    return x_s_all_next, v_s_all_next

In [17]:
def do_simulation(cx, cy, cyaw, ck, sp, dl, initial_state, initial_state_surrounding):
    """
    Simulation

    cx: course x position list
    cy: course y position list
    cy: course yaw position list
    ck: course curvature list
    sp: speed profile
    dl: course tick [m]

    """

    goal = [cx[-1], cy[-1]]

    state = initial_state

    # initial yaw compensation
    if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
    elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0

    time = 0.0
    x = [state.x]
    y = [state.y]
    yaw = [state.yaw]
    v = [state.v]
    t = [0.0]
    d = [0.0]
    a = [0.0]
    target_ind, _ = calc_nearest_index(state, cx, cy, cyaw, 0) # start with 0

    # TODO initial states
    xs_all = []
    vs_all = []
    ys_all = []
    xs = []
    vs = []
    ys = []
    for i_surrounding_vehicle in initial_state_surrounding:
        xs.append(i_surrounding_vehicle[1])
        vs.append(i_surrounding_vehicle[0] + TARGET_SPEED)
        ys.append(i_surrounding_vehicle[3])
    # print(xs)

    odelta, oa = None, None

    cyaw = smooth_yaw(cyaw)

    plt.figure(figsize=(14, 4))
    while MAX_TIME >= time:

        xref, target_ind, dref = calc_ref_trajectory(
            state, cx, cy, cyaw, ck, sp, dl, target_ind)

        x0 = [state.x, state.y, state.v, state.yaw]  # current state

        oa, odelta, ox, oy, oyaw, ov, xbar = iterative_linear_mpc_control(
            xref, x0, dref, oa, odelta)

        if odelta is not None:
            di, ai = odelta[0], oa[0]

        state = update_state(state, ai, di)
        time = time + DT

        # TODO surrounding vehicle update
        xs, vs = IDM_model(xs, vs, state)

        # TODO surrounding
        xs_all.append(xs)
        vs_all.append(vs)
        ys_all.append(ys)

        # ego vehicle
        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        if check_goal(state, goal, target_ind, len(cx)):
            print("Goal")
            break

        if show_animation:  # pragma: no cover
            plt.cla()
            # plot road
            plt.plot([-20, 120], [-4, -4], '-k')
            plt.plot([-20, 120], [4, 4], '-k')
            plt.plot([-20, 120], [0, 0], '--k')

            plt.axis([-20, 120, -30, 30])

            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-.r", label="ref. path")
            plt.plot([-20, 0], [-2, -2], "-.r", label="ref. path")

            plt.plot(x, y, ".m", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(xbar[0, :], xbar[1, :], ".y", label="xbar")
            plt.plot(cx[target_ind], cy[target_ind], "xg", label="target")
            plot_car(state.x, state.y, state.yaw, steer=di)

            # Plot surrounding vehicles
            for i_car in range(len(xs)):
                plot_car(xs[i_car], ys[i_car], 0, steer=0, cabcolor="-b", truckcolor="-b")
                # plt.text(xs[i_car]-2, ys[i_car]-4, 'veh_sur' + str(i_car))
                plt.text(xs[i_car] - 2, ys[i_car] - 4, 'vehicle ' + str(i_car))
                plt.text(xs[i_car] - 2, ys[i_car] - 6, "speed [km/h]:" + str(round(vs[i_car] * 3.6, 2)))

            plt.axis("equal")
            plt.grid(True)
            # plt.legend()

            plt.title("Model Predictive Control Simulation")
            plt.text(-20, 6, "Time[s]:" + str(round(time, 2)))
            plt.text(0, 6, "v_ego [km/h]:" + str(round(state.v * 3.6, 2)))
            # for i_sur, v_sur in enumerate(vs):
            #     plt.text(30 * (i_sur+1), 10, "v_sur" + str(i_sur) + " [km/h]:" + str(round(v_sur * 3.6, 2)))
            plt.pause(0.0001)

    return t, x, y, yaw, v, d, a